In [8]:
# Install required libraries (if not already installed)
%pip install torch torchvision pandas matplotlib opencv-python Pillow

# Import necessary libraries
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.8 MB 1.9 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/7.8 MB 1.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/7.8 MB 1.7 MB/s eta 0:00:04
   ------ --------------------------------- 1.3/7.8 MB 1.1 MB/s eta 0:00:06
   --------- ------------------------------ 1.8/7.8 MB 1.4 MB/s eta 0:00:05
   ------------ --------------------------- 2.4/7.8 MB 1.6 MB/s eta 0:00:04
   ------------ --------------------------- 2.4/7.8 MB 1.6 MB/s eta 0:00:04
   ------------- -------------------------- 2.6/7.8 MB 1.3 MB/s eta 0:00:04
   -------------- ------------------------- 2.9/7.8 MB 1.4 MB/s eta 0:00:04
   ------------------ --------------------- 3.7/7.8 MB 1.6 MB/s eta 0:00:03
   ---------------------- ------------

In [9]:
import pandas as pd

# Load CSV files
train_df = pd.read_csv('dataset/train.csv')
val_df = pd.read_csv('dataset/val.csv')
test_df = pd.read_csv('dataset/test.csv')

# Preview the structure
print(train_df.head())


                     image:FILE  category
0  train/aloevera/aloevera0.jpg         0
1  train/aloevera/aloevera1.jpg         0
2  train/aloevera/aloevera2.jpg         0
3  train/aloevera/aloevera3.jpg         0
4  train/aloevera/aloevera4.jpg         0


In [29]:
class PlantDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        """
        Custom PyTorch Dataset for loading plant images.
        Args:
        - dataframe: Pandas DataFrame with columns [image:FILE, category]
        - root_dir: Base directory containing the images (e.g., 'dataset/')
        - transform: Torchvision transforms for data augmentation
        """
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Combine root_dir and image:FILE column to get full path
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])  # First column = image path
        label = self.data.iloc[idx, 1]  # Second column = category (numeric)

        # Load the image
        image = Image.open(img_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, label


In [30]:
# Load CSV files into dataframes
train_df = pd.read_csv('dataset/train.csv')
val_df = pd.read_csv('dataset/val.csv')
test_df = pd.read_csv('dataset/test.csv')

# Create datasets
train_dataset = PlantDataset(dataframe=train_df, root_dir='dataset/train', transform=transform)
val_dataset = PlantDataset(dataframe=val_df, root_dir='dataset/val', transform=transform)
test_dataset = PlantDataset(dataframe=test_df, root_dir='dataset/test', transform=transform)


In [32]:
print(train_dataset)

In [34]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load CSV files into dataframes
train_df = pd.read_csv('dataset/train.csv')
val_df = pd.read_csv('dataset/val.csv')
test_df = pd.read_csv('dataset/test.csv')

# Create datasets
train_dataset = PlantDataset(dataframe=train_df, root_dir='dataset/', transform=transform)
val_dataset = PlantDataset(dataframe=val_df, root_dir='dataset/', transform=transform)
test_dataset = PlantDataset(dataframe=test_df, root_dir='dataset/', transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [35]:
# List of class names in the order 0-29
class_names = [
    "aloevera", "banana", "bilimbi", "cantaloupe", "cassava", "coconut", "corn", "cucumber",
    "curcuma", "eggplant", "galangal", "ginger", "guava", "kale", "longbeans", "mango", 
    "melon", "orange", "paddy", "papaya", "peperchili", "pineapple", "pomelo", "shallot",
    "soybeans", "spinach", "sweetpotatoes", "tobacco", "waterapple", "watermelon"
]

# Create a mapping of class IDs (0-29) to names
class_mapping = {i: name for i, name in enumerate(class_names)}

# Display the mapping
print(class_mapping)


{0: 'aloevera', 1: 'banana', 2: 'bilimbi', 3: 'cantaloupe', 4: 'cassava', 5: 'coconut', 6: 'corn', 7: 'cucumber', 8: 'curcuma', 9: 'eggplant', 10: 'galangal', 11: 'ginger', 12: 'guava', 13: 'kale', 14: 'longbeans', 15: 'mango', 16: 'melon', 17: 'orange', 18: 'paddy', 19: 'papaya', 20: 'peperchili', 21: 'pineapple', 22: 'pomelo', 23: 'shallot', 24: 'soybeans', 25: 'spinach', 26: 'sweetpotatoes', 27: 'tobacco', 28: 'waterapple', 29: 'watermelon'}


In [36]:
class PlantClassifier(nn.Module):
    def __init__(self, num_classes):
        super(PlantClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PlantClassifier(num_classes=30).to(device)


In [37]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [38]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/10, Loss: 2.6888
Epoch 2/10, Loss: 1.8638
Epoch 3/10, Loss: 1.1352
Epoch 4/10, Loss: 0.6213
Epoch 5/10, Loss: 0.3741
Epoch 6/10, Loss: 0.2534
Epoch 7/10, Loss: 0.1873
Epoch 8/10, Loss: 0.1498
Epoch 9/10, Loss: 0.1517
Epoch 10/10, Loss: 0.1321


In [39]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 68.03%


In [40]:
model.eval()
sample_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        for i in range(len(labels)):
            sample_predictions.append({
                "True Label": class_mapping[labels[i].item()],
                "Predicted": class_mapping[predicted[i].item()]
            })
        break  # Just preview one batch

# Display sample predictions
for pred in sample_predictions[:5]:  # Show first 5 predictions
    print(pred)


{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'spinach'}
{'True Label': 'aloevera', 'Predicted': 'coconut'}
{'True Label': 'aloevera', 'Predicted': 'spinach'}


In [ ]:
torch.save(model.state_dict(), 'plant_classifier.pth2')
print("Model saved as plant_classifier.pth")


Model saved as plant_classifier.pth


In [42]:
model.load_state_dict(torch.load('plant_classifier.pth'))
model.eval()
print("Model loaded successfully.")


Model loaded successfully.

C:\Users\anirv\AppData\Local\Temp\ipykernel_11180\3744957548.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('plant_classifier.pth'))


In [43]:
model.eval()
correct, total = 0, 0
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 61.88%


In [44]:
for pred in sample_predictions[:100]:  # Show first 5 predictions
    print(pred)

{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'spinach'}
{'True Label': 'aloevera', 'Predicted': 'coconut'}
{'True Label': 'aloevera', 'Predicted': 'spinach'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'peperchili'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'mango'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'soybeans'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': 'aloevera', 'Predicted': 'aloevera'}
{'True Label': '

In [2]:
import torch
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>